<a href="https://colab.research.google.com/github/dacado0122-create/Colab/blob/main/codigos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**codigo 1.1**

In [7]:

import numpy as np

# Parámetros del problema
D = 1e-3
r = 1.0
N = 101        # puntos de malla en x,y
T = 1.0        # tiempo final

# Malla espacial
x = np.linspace(0, 1, N)
y = np.linspace(0, 1, N)
dx = x[1] - x[0]

# Condición CFL
dt_max = dx**2 / (4 * D)
dt = 1e-3  # seguro y cumple CFL
assert dt < dt_max, "dt NO cumple CFL"

Nt = int(T / dt)

# Inicializar solución
X, Y = np.meshgrid(x, y, indexing='ij')
u = np.exp(-((X - 0.5)**2 + (Y - 0.5)**2) / 0.05)

# Tensor de salida
Ground_Truth = np.zeros((Nt, N, N))
Ground_Truth[0] = u.copy()

def laplaciano_periodico(u):
    """Implementa el stencil de 5 puntos con fronteras periódicas."""
    return (
        np.roll(u, 1, axis=0) + np.roll(u, -1, axis=0) +
        np.roll(u, 1, axis=1) + np.roll(u, -1, axis=1) -
        4 * u
    ) / dx**2

# Evolución temporal
for n in range(1, Nt):
    lap = laplaciano_periodico(u)
    reaction = r * u * (1 - u)
    u = u + dt * (D * lap + reaction)
    Ground_Truth[n] = u.copy()

print("Ground_Truth shape:", Ground_Truth.shape)


Ground_Truth shape: (1000, 101, 101)


**codigo 1.2**

In [ ]:
import torch
import torch.nn as nn
import numpy as np

# -----------------------------
# 1. Cargar datos Ground Truth
# -----------------------------
GT = Ground_Truth  # (Nt, Nx, Ny)
Nt, Nx, Ny = GT.shape

# Crear malla de coordenadas
x = np.linspace(0, 1, Nx)
y = np.linspace(0, 1, Ny)
t = np.linspace(0, 1, Nt)

X, Y, T = np.meshgrid(x, y, t, indexing='ij')

coords = np.stack(
    [X.flatten(), Y.flatten(), T.flatten()],
    axis=1
)

values = GT.transpose(1, 2, 0).flatten()  # u(x,y,t)

# Pasar a tensores PyTorch
coords = torch.tensor(coords, dtype=torch.float32)
values = torch.tensor(values, dtype=torch.float32).unsqueeze(1)

# -----------------------------
# 2. Definir red neuronal
# -----------------------------
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.net(x)

model = DNN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# -----------------------------
# 3. Entrenamiento
# -----------------------------
batch_size = 1024
n_epochs = 2000

dataset = torch.utils.data.TensorDataset(coords, values)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True
)

for epoch in range(n_epochs):
    for xb, yb in dataloader:
        pred = model(xb)
        loss = loss_fn(pred, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 200 == 0:
        print(f"Epoch {epoch} - Loss: {loss.item():.6f}")

# -----------------------------
# 4. Evaluación / extrapolación
# -----------------------------
# Punto fuera del tiempo de entrenamiento (t > T)
test_point = torch.tensor([[0.2, 0.8, 1.2]], dtype=torch.float32)
print("Extrapolación:", model(test_point).item())


Epoch 0 - Loss: 0.000033


**codigo 2.1**

In [ ]:
import torch
import torch.nn as nn

# -----------------------------
# 1. Definición del PINN
# -----------------------------
class PINN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(3, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        return self.net(x)

model = PINN()

# -----------------------------
# 2. Derivación automática
# -----------------------------
def gradients(u, x):
    """Calcula du/dx para un escalar u y vector x=[x,y,t]."""
    return torch.autograd.grad(
        u, x,
        grad_outputs=torch.ones_like(u),
        create_graph=True,
        retain_graph=True
    )[0]

def laplacian(u, x):
    """Devuelve u_xx + u_yy mediante autograd."""
    grads = gradients(u, x)
    ux, uy, ut = grads[:, 0], grads[:, 1], grads[:, 2]

    ux_x = torch.autograd.grad(
        ux, x,
        grad_outputs=torch.ones_like(ux),
        create_graph=True
    )[0][:, 0]

    uy_y = torch.autograd.grad(
        uy, x,
        grad_outputs=torch.ones_like(uy),
        create_graph=True
    )[0][:, 1]

    return ux_x + uy_y, ut

# -----------------------------
# 3. Pérdida del residuo
# -----------------------------
D = 1e-3
r = 1.0

def residual_loss(model, Xf):
    Xf.requires_grad_(True)
    u = model(Xf)
    lap_u, u_t = laplacian(u, Xf)
    res = u_t - D * lap_u - r * u * (1 - u)
    return torch.mean(res**2)

# -----------------------------
# 4. Pérdida de condición inicial
# -----------------------------
def ic_loss(model, X0, U0):
    u_pred = model(X0)
    return torch.mean((u_pred - U0)**2)

==

# Puntos de colocación en el dominio
Nf = 5000
Xf = torch.rand(Nf, 3)

# Puntos para condición inicial (t = 0)
N0 = 1000
X0 = torch.rand(N0, 3)
X0[:, 2] = 0.0

# Condición inicial
U0 = torch.exp(
    -((X0[:, 0] - 0.5)**2 + (X0[:, 1] - 0.5)**2) / 0.05
).unsqueeze(1)

# -----------------------------
# 5. Entrenamiento
# -----------------------------
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(5000):
    loss_res = residual_loss(model, Xf)
    loss_ic = ic_loss(model, X0, U0)
    loss = loss_res + loss_ic

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
        print(f"Epoch {epoch} | Loss: {loss.item():.6f}")


**codigo 3**

In [ ]:
import torch
import torch.nn as nn

# -----------------------------
# 1. Definición del bloque PDE-Net
# -----------------------------
class PDENet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=16,
            kernel_size=3, padding=1, padding_mode='circular'
        )
        self.conv2 = nn.Conv2d(
            in_channels=16, out_channels=1,
            kernel_size=3, padding=1, padding_mode='circular'
        )
        self.activation = nn.ReLU()

        # Inicialización física del primer kernel
        laplace = torch.tensor([
            [0., 1., 0.],
            [1., -4., 1.],
            [0., 1., 0.]
        ])

        # Ajustar dimensiones: out_channels x in_channels x 3 x 3
        self.conv1.weight.data[0, 0, :, :] = 0.01 * laplace

    def forward(self, U):
        x = self.activation(self.conv1(U))
        x = self.conv2(x)
        return U + x  # bloque residual

# -----------------------------
# 2. Preparación de datos
# -----------------------------
# Ground_Truth: (Nt, Nx, Ny)
GT_tensor = torch.tensor(Ground_Truth, dtype=torch.float32)
GT_tensor = GT_tensor.unsqueeze(1)  # -> (Nt, 1, Nx, Ny)

U_t = GT_tensor[:-1]    # (Nt-1, 1, Nx, Ny)
U_tp1 = GT_tensor[1:]  # (Nt-1, 1, Nx, Ny)

dataset = torch.utils.data.TensorDataset(U_t, U_tp1)
loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=8,
    shuffle=True
)

# -----------------------------
# 3. Entrenamiento
# -----------------------------
model = PDENet()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

for epoch in range(2000):
    for u, target in loader:
        pred = model(u)
        loss = loss_fn(pred, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 200 == 0:
        print(f"Epoch {epoch} - Loss: {loss.item():.6e}")


**codigo 4**

In [ ]:
import torch
import torch.nn as nn

class PatchEmbedding(nn.Module):
    def __init__(self, patch_size, emb_dim):
        super().__init__()
        self.patch_size = patch_size
        self.proj = nn.Linear(patch_size * patch_size, emb_dim)

    def forward(self, x):
        # x: (B, 1, N, N)
        B, _, N, _ = x.shape
        P = self.patch_size
        x = x.unfold(2, P, P).unfold(3, P, P)
        x = x.contiguous().view(B, -1, P * P)
        return self.proj(x)

class SimpleTransformer(nn.Module):
    def __init__(self, patch_size=8, emb_dim=128, n_heads=4):
        super().__init__()
        self.embed = PatchEmbedding(patch_size, emb_dim)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=emb_dim,
            nhead=n_heads,
            batch_first=True
        )

        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=4
        )

        self.fc = nn.Linear(emb_dim, patch_size * patch_size)
        self.patch_size = patch_size

    def forward(self, U):
        Z = self.embed(U)
        Z = self.transformer(Z)
        Z = self.fc(Z)

        # reconstrucción
        B, NP, _ = Z.shape
        P = self.patch_size
        N = int((NP) ** 0.5)

        Z = Z.view(B, N, N, P, P)
        Z = Z.permute(0, 1, 3, 2, 4).contiguous()

        return Z.view(B, 1, N * P, N * P)


**codigo 5.1**

In [ ]:
import torch
import torch.nn as nn

class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.conv = nn.Conv2d(
            input_dim + hidden_dim,
            4 * hidden_dim,
            kernel_size,
            padding=kernel_size // 2
        )

    def forward(self, x, hidden):
        h_cur, c_cur = hidden

        combined = torch.cat([x, h_cur], dim=1)
        gates = self.conv(combined)

        i, f, o, g = torch.split(gates, self.hidden_dim, dim=1)

        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next


class ConvLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers):
        super().__init__()

        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.layers = nn.ModuleList([
            ConvLSTMCell(
                input_dim if i == 0 else hidden_dim,
                hidden_dim,
                kernel_size
            )
            for i in range(num_layers)
        ])

    def forward(self, x_seq):
        batch_size, seq_len, _, height, width = x_seq.shape

        h = [
            torch.zeros(batch_size, self.hidden_dim, height, width).to(x_seq.device)
            for _ in range(self.num_layers)
        ]
        c = [
            torch.zeros(batch_size, self.hidden_dim, height, width).to(x_seq.device)
            for _ in range(self.num_layers)
        ]

        outputs = []

        for t in range(seq_len):
            x = x_seq[:, t]

            for i, layer in enumerate(self.layers):
                h[i], c[i] = layer(x, (h[i], c[i]))
                x = h[i]

            outputs.append(h[-1])

        return torch.stack(outputs, dim=1)


**codigo 5.2**

In [ ]:
def variational_loss(model, X, T, k=1):
    X.requires_grad_(True)

    u = model(X)

    grads = torch.autograd.grad(
        u, X,
        grad_outputs=torch.ones_like(u),
        create_graph=True
    )[0]

    ux, uy, ut = grads[:, 0], grads[:, 1], grads[:, 2]
    x, y = X[:, 0], X[:, 1]

    v = torch.sin(k * torch.pi * x) * torch.sin(k * torch.pi * y)
    vx = k * torch.pi * torch.cos(k * torch.pi * x) * torch.sin(k * torch.pi * y)
    vy = k * torch.pi * torch.sin(k * torch.pi * x) * torch.cos(k * torch.pi * y)

    integrand = ut * v + D * (ux * vx + uy * vy) - r * u * (1 - u) * v

    volume = T
    integral = volume * torch.mean(integrand)

    return integral ** 2
